## 模擬槍枝射擊情況

In [28]:
import pygame
import pandas as pd
import numpy as np
import json

gun_json_path = r"C:\Users\landis\Desktop\MDPI\ArUco\marker_19_position.json"
monkey_csv_path = r"C:\Users\landis\Desktop\MDPI\random_positions.csv"
camera_json_path = r"C:\Users\landis\Desktop\MDPI\camera_extrinsics.json"

# 讀取資料
with open(gun_json_path, "r") as f:
    gun_data = json.load(f)
gun_pos = np.array(gun_data["position"])

df = pd.read_csv(monkey_csv_path)
monkey_positions = df[["X", "Y", "Z"]].dropna().values

with open(camera_json_path, "r") as f:
    cam_data = json.load(f)
cam_positions = [np.array(cam_data[k]["world_position"]) for k in cam_data]

# 加入誤差並預先儲存預測位置
mae_std = 0.124174
predicted_positions = monkey_positions + np.random.normal(0.0, mae_std, monkey_positions.shape)


# pygame 初始化
pygame.init()
WIDTH, HEIGHT = 1200, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("漆彈槍 射擊模擬（含相機位置與角度）")
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 24)

def add_prediction_noise(pos, mae_std=0.124174):
    noise = np.random.normal(loc=0.0, scale=mae_std, size=3)
    return pos + noise

def world_to_screen_xy(pos):
    scale = 100
    x = int(WIDTH / 4 + pos[0] * scale)
    y = int(HEIGHT / 2 - pos[1] * scale)
    return (x, y)

def world_to_screen_xz(pos):
    scale = 100
    x = int(3 * WIDTH / 4 + pos[0] * scale)
    y = int(HEIGHT / 2 - pos[2] * scale)
    return (x, y)

def get_direction_vector(from_pos, to_pos):
    dir = to_pos - from_pos
    norm = np.linalg.norm(dir)
    return dir / norm if norm > 0 else dir

def check_hit(bullet_pos, monkey_pos, radius=0.25):
    return np.linalg.norm(bullet_pos - monkey_pos) < radius

def draw_text(surface, text, x, y, color=(0, 0, 0)):
    img = font.render(text, True, color)
    surface.blit(img, (x, y))

# 初始變數
index = 0
bullet_t = 0
speed = 0.4
bullet_active = True
total_shots = 0
hit_count = 0

# 主迴圈
running = True
while running:
    if index >= len(monkey_positions):
        break

    screen.fill((255, 255, 255))

    # 目前目標座標與方向
    true_pos = monkey_positions[index]  # 真實位置
    target_pos = predicted_positions[index]# 加入 MAE 模擬誤差
    direction = get_direction_vector(gun_pos, target_pos) # 槍朝向預測結果

    # 計算角度
    delta = target_pos - gun_pos
    azimuth_deg = np.degrees(np.arctan2(delta[0], delta[1]))
    elevation_deg = np.degrees(np.arctan2(delta[2], np.linalg.norm(delta[:2])))

    # 左邊 XY 平面
    for cam in cam_positions:
        # 左邊 XY 視圖框線:
        pygame.draw.rect(screen, (0, 0, 0), (0, 0, WIDTH // 2, HEIGHT), 2)
        pygame.draw.circle(screen, (255, 165, 0), world_to_screen_xy(cam), 5)  # 橘色相機
        pygame.draw.circle(screen, (0, 255, 0), world_to_screen_xy(gun_pos), 6)     # 綠色槍口

        # 真實猴子位置（紅色空心圈）
        pygame.draw.circle(screen, (255, 0, 0), world_to_screen_xy(true_pos), 10, width=2)
        # 預測猴子位置（藍色實心）
        pygame.draw.circle(screen, (0, 0, 255), world_to_screen_xy(target_pos), 6)


    # 右邊 XZ 平面
    for cam in cam_positions:
        # 右邊 XZ 視圖框線
        pygame.draw.rect(screen, (0, 0, 0), (WIDTH // 2, 0, WIDTH // 2, HEIGHT), 2)
        pygame.draw.circle(screen, (255, 165, 0), world_to_screen_xz(cam), 5)
        pygame.draw.circle(screen, (0, 255, 0), world_to_screen_xz(gun_pos), 6)

        # 真實猴子位置（紅色空心圈）
        pygame.draw.circle(screen, (255, 0, 0), world_to_screen_xz(true_pos), 10, width=2)
        # 預測猴子位置（藍色實心）
        pygame.draw.circle(screen, (0, 0, 255), world_to_screen_xz(target_pos), 6)


    # 子彈
    if bullet_active:
        bullet_pos = gun_pos + direction * bullet_t
        pygame.draw.circle(screen, (0, 0, 0), world_to_screen_xy(bullet_pos), 5)
        pygame.draw.circle(screen, (0, 0, 0), world_to_screen_xz(bullet_pos), 5)
        bullet_t += speed

        if check_hit(bullet_pos, true_pos):
            # 顯示紅色實心圓表示命中真實猴子
            pygame.draw.circle(screen, (255, 0, 0), world_to_screen_xy(true_pos), 12)
            pygame.draw.circle(screen, (255, 0, 0), world_to_screen_xz(true_pos), 12)

            bullet_active = False
            hit_count += 1
            total_shots += 1

        elif bullet_t > 30:
            bullet_active = False
            total_shots += 1

    # 下一筆
    if not bullet_active:
        index += 1
        bullet_t = 0
        bullet_active = index < len(monkey_positions)

    # 角度與命中率顯示
    # 圖例（左下角）
    draw_text(screen, "blue pre pos", 20, HEIGHT - 70, (0, 0, 255))
    draw_text(screen, "red true pos", 20, HEIGHT - 50, (255, 0, 0))
    draw_text(screen, "black bullet", 20, HEIGHT - 30, (0, 0, 0))

    draw_text(screen, f"Azimuth: {azimuth_deg:.2f}°", 20, 20)
    draw_text(screen, f"Elevation: {elevation_deg:.2f}°", 20, 50)
    if total_shots > 0:
        draw_text(screen, f"Hit: {hit_count}/{total_shots} ({(hit_count / total_shots * 100):.2f}%)", 20, 80)

    pygame.display.flip()
    clock.tick(600)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

pygame.quit()

if total_shots > 0:
    hit_rate = hit_count / total_shots * 100
    print(f"\n模擬結束")
    print(f"總射擊次數: {total_shots}")
    print(f"命中次數: {hit_count}")
    print(f"命中率: {hit_rate:.2f}%")
else:
    print("沒有有效射擊")



模擬結束
總射擊次數: 100
命中次數: 82
命中率: 82.00%


In [27]:
import pandas as pd
import numpy as np

# 參數設定
num_samples = 100  #資料筆數
x_range = (-0.7, 0.07)
y_range = (0.13, 0.24)
z_range = (0.59, 3.03)

# 生成亂數 
x_values = np.random.uniform(x_range[0], x_range[1], num_samples)
y_values = np.random.uniform(y_range[0], y_range[1], num_samples)
z_values = np.random.uniform(z_range[0], z_range[1], num_samples)

# 建立 DataFrame
df = pd.DataFrame({
    "X": x_values,
    "Y": y_values,
    "Z": z_values
})

# 儲存成 CSV
df.to_csv("random_positions.csv", index=False)

print("已產生隨機資料並儲存為 random_positions.csv")


已產生隨機資料並儲存為 random_positions.csv
